In [3]:
import sys
sys.path.insert(0, f'D:/OneDrive/Program/GitHub/EasyIDP')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from easyric.objects import Pix4D
from easyric.io import shp, geotiff, plot
from easyric.calculate import geo2raw, geo2tiff

In [5]:
project_name = 'broccoli_tanashi_5_20200528_P4M_10m_before'
project_path = f"Z:/共享云端硬盘/broccoliProject/tanashi_broccoli2020_RGB_AP/{project_name}"
raw_img_folder=f"Z:/共享云端硬盘/broccoliProject/rgb_raw/{project_name}"
param_folder = f"{project_path}/params"

In [6]:
p4d = Pix4D(project_path=project_path, 
            raw_img_path=raw_img_folder, 
            project_name=project_name,
            param_folder=param_folder)

[Init][Pix4D] No ply given, however find 'broccoli_tanashi_5_20200528_P4M_10m_before_group1_densified_point_cloud.ply' at current project folder
[Init][Pix4D] No dom given, however find 'broccoli_tanashi_5_20200528_P4M_10m_before_transparent_mosaic_group1.tif' at current project folder
[Init][Pix4D] No dsm given, however find 'broccoli_tanashi_5_20200528_P4M_10m_before_dsm.tif' at current project folder
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]


# calculate the w and h of grid

Almost 50 per image

<img src="https://img.sigmameow.com/picgo/broccoli_20210121095943.png" />

In [7]:
x0 = 368080.160; y0 = 3956033.527
x1 = 368083.927; y1 = 3956030.399

In [10]:
px = geotiff.geo2pixel(points_hv = np.asarray([[x0, y0], [x1, y1]]), geo_head=p4d.dom_header)
px

array([[ 9466, 13989],
       [10866, 15152]])

In [12]:
(px[1,:] - px[0,:]).mean()

1281.5

In [22]:
from easyric.caas_lite import TiffSpliter
ts = TiffSpliter(tif_path=p4d.dom_file, grid_h=1300, grid_w=1300, grid_buffer=200)

In [23]:
ts.grid_h, ts.grid_w

(1300, 1300)

In [24]:
ts.wgrid_len

array([1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300,
       1300, 1300, 1300, 1300, 1300,  491], dtype=uint32)

In [25]:
ts.save_one_grid('out/01_grid_x4_y4.tif', w_id=4, h_id=4)

True

In [26]:
ts.save_one_grid('out/01_grid_x5_y5.tif', w_id=5, h_id=5)

True

In [34]:
ts.save_one_grid('out/01_grid_x1_y1.tif', w_id=1, h_id=1)

False

# save offset json

```
##########################
#              x         #
# North O------------->  #
#       |  ________      #
#       | |        |     #
#       | |  img   |     #
#       | |________|     #
#       v                #
##########################

x: horizontal,  width, longitude
y:   vertical, height,  latitude
```

In [27]:
ts.wgrid_st

array([    0,  1300,  2600,  3900,  5200,  6500,  7800,  9100, 10400,
       11700, 13000, 14300, 15600, 16900, 18200, 19500, 20800])

In [28]:
ts.wgrid_len

array([1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300, 1300,
       1300, 1300, 1300, 1300, 1300,  491], dtype=uint32)

In [31]:
offset_json = {}
for w_id, w_st in enumerate(ts.wgrid_st):
    for h_id, h_st in enumerate(ts.hgrid_st):
        tiff_name = ts.id2name(w_id=w_id, h_id=h_id)
        offset_json[tiff_name] = {'x':ts.wgrid_st[w_id], 
                                  'y':ts.hgrid_st[h_id]}

In [32]:
ts.dict2json(offset_json, 'out/01_offset.json')

# id digits

In [39]:
int('00')

0

In [48]:
'{0:04}'.format(1, 3)

'0001'

In [54]:
f"{{0:0{3}}}"

'{0:03}'

In [53]:
f"{{0:0{3}}}".format(3)

'003'